# import def func

In [1]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [35]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

from JSONData import tdx_hdf5_api as h5a
def get_tdx_all_from_h5(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d':
        dl='60'
    else:
        dl='200'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


def get_tdx_all_from_h5(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
    if end is not None:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
            '_' + end.replace('-', '') + '_' + 'all'
    else:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append_status,
                                      doubleFile=False, keep_last=0)
                ("write:%s block_path:%s" % (len(codew), block_path))
            else:
                ("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, False,
                                      doubleFile=False, keep_last=0)
                ("write:%s block_path:%s" % (len(codew), block_path))
        else:
            ("write error:%s block_path:%s" % (len(codew), block_path))
    return True

## init data

### get_tdx_all

In [3]:
h5_day=get_tdx_all_from_h5(resample='d')

keys:['/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


### sina

In [4]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-18',
 code
 832876    慧为智能
 Name: name, dtype: object)

In [5]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/19/25 00:02:55'

code
600355   2025-06-19 15:00:03
601188   2025-06-19 15:00:03
Name: ticktime, dtype: datetime64[ns]

-897.1276809096337

#### 实时成交

In [12]:
def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [14]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:G:\sina_MultiIndex_data.h5
[]
G:\sina_MultiIndex_data.h5


#### 读取实时成交个股

In [ ]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

##### 合并实时成交量及SinaDM

In [15]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### resample 3d

In [19]:
h5_3d = get_tdx_all_from_h5(resample='3d')
tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [20]:
tdx_3d.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
show_tdx_data(tdx_select)

(5183, 211)

Count:0 211


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,


In [21]:
t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')

In [22]:
show_tdx_data(t3d_code)

Count:342 211


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,
000001,平安银行,2025-04-07,11.77,11.45,10.12,0.10,10.12,1.3,0.1
000159,国际实业,2025-04-10,5.88,5.68,4.21,1.20,4.21,0.0,1.3
000407,胜利股份,2025-04-07,3.56,3.38,2.76,30.00,2.98,0.0,0.1
000408,藏格矿业,2025-04-07,40.91,37.59,31.00,0.00,32.53,20.3,7.3
000430,ST张家界,2025-04-25,8.37,9.17,6.60,25.59,6.60,22.3,1.2
000519,中兵红箭,2025-04-07,17.55,18.05,15.17,1.20,15.17,-0.1,-0.1
000520,凤凰航运,2025-04-07,5.54,5.69,3.44,25.41,4.08,1.2,1.2
000543,皖能电力,2025-04-07,7.49,7.76,6.95,1.20,6.95,-0.1,0.0
000554,泰山石油,2025-04-07,7.76,6.46,5.00,35.59,5.54,0.1,1.3


### resample W

In [25]:
h5_w = get_tdx_all_from_h5(resample='w')
tdx_w = cct.combine_dataFrame(h5_w,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [26]:
tdx_w.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
tdx_select_w = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')[:10]
show_tdx_data(tdx_select_w)

(5186, 213)

Count:10 213


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,
600156,NaN,2025-01-10,NaN,6.39,4.05,1.20,4.31,21.70,1.2
000001,平安银行,2025-04-11,11.77,11.61,10.12,1.30,10.12,1.30,1.2
000408,藏格矿业,2024-12-20,40.91,36.81,26.75,1.20,26.75,0.00,0.0
000553,安道麦A,2025-04-18,7.10,7.27,5.22,-1.00,5.38,1.20,16.8
000631,顺发恒业,2025-04-11,2.98,3.42,2.43,1.20,2.43,1.20,-1.1
000657,中钨高新,2025-01-03,12.05,11.53,8.50,0.00,8.70,31.30,-0.9
000663,永安林业,2025-01-10,6.56,6.67,4.71,0.00,4.82,1.10,1.3
000701,厦门信达,2025-01-03,6.53,5.70,3.71,1.20,4.30,33.59,1.2
000766,通化金马,2024-11-29,27.98,20.34,13.59,45.59,13.59,26.41,1.2


In [27]:
tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
show_tdx_data(tw_code)

Count:628 213


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,
000001,平安银行,2025-04-11,11.77,11.61,10.12,1.30,10.12,1.3,1.20
000020,深华发Ａ,2025-04-11,15.16,14.71,9.22,51.59,9.22,1.1,1.20
000049,德赛电池,2025-04-11,21.67,24.81,17.91,0.00,17.91,0.0,0.00
000159,国际实业,2025-04-11,5.88,7.67,4.21,1.20,4.21,4.1,-0.10
000338,潍柴动力,2024-12-20,15.50,16.59,12.97,1.20,13.07,0.1,0.00
000407,胜利股份,2025-04-11,3.56,3.72,2.76,25.30,2.76,0.1,-0.10
000408,藏格矿业,2024-12-20,40.91,36.81,26.75,1.20,26.75,0.0,0.00
000428,华天酒店,2025-01-10,3.30,3.94,2.73,0.00,2.90,1.3,-0.10
000430,ST张家界,2025-01-03,8.37,9.17,5.56,1.20,5.80,1.2,0.10


### resample M

In [28]:
h5_m = get_tdx_all_from_h5(resample='m')
tdx_m = cct.combine_dataFrame(h5_m,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [29]:
tm_code = tdx_m.query('lasth1d > lasth2d > lasth3d and close >= high4')
show_tdx_data(tm_code)

Count:792 211


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,
000001,平安银行,2025-04-30,11.77,11.60,9.00,1.30,10.12,-1.1,1.20
000010,美丽生态,2024-09-30,3.05,3.03,1.38,1.20,1.64,-1.1,1.10
000017,深中华A,2025-02-28,6.80,7.12,5.06,1.20,5.50,0.0,0.10
000020,深华发Ａ,2025-04-30,15.16,15.72,9.00,1.30,9.22,-1.1,-0.90
000159,国际实业,2025-04-30,5.88,7.44,4.83,1.30,4.21,0.0,1.10
000407,胜利股份,2024-09-30,3.56,3.45,2.47,1.30,2.60,0.0,-0.90
000408,藏格矿业,2024-12-31,40.91,35.84,21.14,36.19,26.75,0.0,1.20
000426,兴业银锡,2024-12-31,16.30,14.21,9.50,45.09,11.10,0.0,0.20
000513,丽珠集团,2025-02-28,35.62,39.56,32.97,1.20,32.97,-1.0,-1.10


### resample D

In [30]:
h5_d = get_tdx_all_from_h5(resample='d')
tdx_d = cct.combine_dataFrame(h5_d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [31]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
td_code = tdx_d.query('ma51d > ma10d and ral > 3 and low >ma10d')
show_tdx_data(td_code,limit=5)

Count:763 213


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,
000001,平安银行,2025-04-07,11.77,11.68,10.53,1.20,10.12,1.2,1.2
000020,深华发Ａ,2025-04-08,15.16,12.85,11.49,32.31,9.85,-0.1,0.0
000027,深圳能源,2025-04-07,6.38,6.44,6.20,0.10,5.64,1.2,0.1
000037,深南电A,2025-04-08,9.39,9.46,8.23,1.30,7.37,1.2,1.3
000100,TCL科技,2025-04-09,4.37,4.42,4.16,1.30,3.68,0.0,1.3


In [32]:
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))

In [33]:
show_tdx_data(td_code.loc[code_f])

Count:123 213


,name,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,
600901,江苏金租,2025-04-11,6.15,6.01,4.98,0.00,4.75,1.30,1.2
600039,四川路桥,2025-03-31,9.88,9.53,8.58,1.20,7.89,1.30,0.1
600331,宏达股份,2025-04-08,7.81,7.67,6.62,1.30,5.95,1.30,0.0
300940,南极光,2025-04-07,25.44,20.27,16.14,20.30,12.55,23.80,0.0
600423,柳化股份,2025-04-08,3.46,3.35,3.06,1.30,2.79,1.30,0.1
001299,美能能源,2025-04-07,11.38,11.27,10.69,30.00,9.41,1.30,0.1
601857,中国石油,2025-04-07,9.18,8.88,7.89,1.30,7.33,1.30,1.3
002828,贝肯能源,2025-04-08,13.43,9.29,8.08,31.09,7.25,25.30,0.1
600908,无锡银行,2025-04-07,6.36,6.28,5.50,1.30,5.22,1.20,0.1


### Write toblk

In [38]:
write_code_to_blk(code_f,'077',rewrite=False)

write blk [Y] or [N]: y
Rewrite code [Y] or append [N](defalut:N): n


all write to D:\MacTools\WinTools\new_tdx2\T0002\blocknew\077.blk:117
counts:0 data:123 :D:\MacTools\WinTools\zd_dxzq\T0002\blocknew\077.blk


True